In [1]:
import seaborn as sns
import pandas as pd
import folium

In [14]:
# dataset 확인
df = pd.read_csv('/Users/joyeongyu/Downloads/취준/데브코스/samsungZZ/서울교통공사_역별 일별 시간대별 승하차인원(24.2~24.5).csv', encoding='cp949')
# 5월 데이터만 남기기
df['날짜'] = pd.to_datetime(df['날짜'], errors='coerce')  # 변환 중 오류 시 NaT로 처리
# 2. 2024년 5월 데이터 필터링
df = df[(df['날짜'].dt.year == 2024) & (df['날짜'].dt.month == 5)]
# 결과 확인
df = df.reset_index()

In [15]:
import re

df['역명'] = df['역명'].apply(lambda x: re.sub(r'\([^)]*\)', '', x) if '(' in x else x)
df['역명'] = df['역명'].apply(lambda x: x+'역' if '서울역' not in x else x)
distinct_station_name =df['역명'].unique()

In [16]:
import requests
x = []
y = []
for z in distinct_station_name:
    url = f'https://dapi.kakao.com/v2/local/search/keyword.json?query={z}'
    headers = {"Authorization": 'KakaoAK 280b770ef839100d3cfbd61f397633c9'}
    places = requests.get(url, headers = headers).json()['documents'][0]
    x.append(places['y'])
    y.append(places['x'])

In [17]:
station_map_dict = {}
for i in range(len(distinct_station_name)):
    station_map_dict[distinct_station_name[i]] = [x[i], y[i]]

In [18]:
for i in range(len(df)):
    df.loc[i, 'latitude'] = station_map_dict[df.loc[i]['역명']][0]
    df.loc[i, 'longitude'] = station_map_dict[df.loc[i]['역명']][1]

In [37]:
time_cols = [ '06시 이전', '06시-07시', '07시-08시', '08시-09시', '09시-10시', '10시-11시', '11시-12시', '12시-13시', '13시-14시',
'14시-15시', '15시-16시', '16시-17시', '17시-18시', '18시-19시', '19시-20시', '20시-21시', '21시-22시', '22시-23시', '23시-24시', '24시 이후']

# 1. 피벗 테이블 생성 (호선, 날짜, 역명 기준으로 집계)
pivot = df.pivot_table(
    index=['날짜', '호선', '역명', 'latitude', 'longitude'],  # 호선을 추가
    columns='구분',
    values=time_cols,
    aggfunc='sum',
    fill_value=0  # NaN 값은 0으로 대체
)

# 피벗 테이블 확인
print(pivot.head())

# 2. 승차-하차 차이 계산
diff = pivot.xs('승차', level=1, axis=1) - pivot.xs('하차', level=1, axis=1)

# 3. 차이 데이터프레임 정리 (호선, 날짜, 역명, 시간대별로)
result = diff.reset_index().melt(id_vars=['날짜', '호선', '역명', 'latitude', 'longitude'], var_name='시간대', value_name='승차_하차_차이')
result['latitude'] = pd.to_numeric(result['latitude'], errors='coerce')
result['longitude'] = pd.to_numeric(result['longitude'], errors='coerce')
result['승차_하차_차이'] = result['승차_하차_차이'].astype(int)
# 4. 결과 출력
print(result.head()) # +는 승차인원이 많은것, -는 하차인원이 많은것

                                                          06시 이전      06시-07시  \
구분                                                            승차   하차      승차   
날짜         호선  역명   latitude           longitude                                
2024-05-01 1호선 동대문역 37.571669405802616 127.01063381083677    388   92     277   
               동묘앞역 37.5732166600283   127.016364227103       87   30     132   
               서울역  37.5546788388674   126.970606917394      187  223     297   
               시청역  37.56534539636417  126.97719821079865     79  114     118   
               신설동역 37.5760299683175   127.024456700382      214   53     242   

                                                                07시-08시        \
구분                                                           하차      승차    하차   
날짜         호선  역명   latitude           longitude                                
2024-05-01 1호선 동대문역 37.571669405802616 127.01063381083677   271     425   466   
               동묘앞역 37.5732

In [39]:
#시각화 준비
import folium

date = '2024-05-01'
line = '1호선'
time = '07시-08시'

# 필터링된 결과
table = result[
    (result['날짜'] == date) & 
    (result['호선'] == line) & 
    (result['시간대'] == time)
]
table = table.reset_index()
# 지도 초기화
map = folium.Map(location=[table['latitude'].mean(), table['longitude'].mean()], zoom_start=13)

# CircleMarker 추가
for i, row in table.iterrows():
    folium.CircleMarker(
        location=(row['latitude'], row['longitude']),  # (위도, 경도)
        radius=abs(row['승차_하차_차이']) / 100,  # 반지름 조정
        popup=row['역명'],
        color='blue' if row['승차_하차_차이'] > 0 else 'red',
        fill=True,
        fill_opacity=0.7
    ).add_to(map)

# 지도 출력
map


In [40]:
map.save('output.html')